# Semantic Routing

In [2]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. You are great at 
    answering questions about physics in a concise and easy-to-understand manner. 
    When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering 
    math questions. You are so good because you are able to break down hard 
    problems into their component parts, answer the component parts, and then 
    put them together to answer the broader question.

Here is a question:
{query}"""

# Embed prompts
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Route question to prompt
@chain
def prompt_router(query):
    # Embed question
    query_embedding = embeddings.embed_query(query)
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    # Pick the prompt most similar to the input question
    most_similar = prompt_templates[similarity.argmax()]
    return PromptTemplate.from_template(most_similar)

semantic_router = (
    prompt_router
    | ChatOllama(model="llama3.1")
    | StrOutputParser()
)

print(semantic_router.invoke("What's a black hole"))

A great question to start with! A black hole is an incredibly dense region of space where gravity is so strong that nothing, including light, can escape once it gets too close. It's formed when a massive star collapses in on itself, causing its gravitational pull to become impossibly strong.

Think of it like this: imagine a vacuum cleaner that sucks up everything around it, but instead of dust and dirt, it sucks up space and time themselves. The point of no return, where anything that crosses the event horizon (the boundary around the black hole) gets trapped forever, is called the singularity. It's a one-way portal to the unknown.

Black holes come in different sizes, ranging from small ones formed by the collapse of individual stars, to supermassive ones at the centers of galaxies, with masses millions or even billions of times that of our sun!


Let's see the next method, called **Query construction**. Click [here](../README.md/#query-construction)